In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [4]:
df = df.sort_values("date")

In [5]:
df = df.reset_index(drop=True)

In [7]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [8]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team
    
df = df.groupby("team", group_keys=False).apply(add_target)

/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/4107492548.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/4107492548.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/4107492548.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [9]:
df["target"][pd.isnull(df["target"])] = 2

/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/1618397414.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/161839

In [11]:
df["target"] = df["target"].astype(int, errors="ignore")

In [13]:
nulls = pd.isnull(df)

In [14]:
nulls = nulls.sum()

In [15]:
nulls = nulls[nulls > 0]

In [16]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [20]:
df = df[valid_columns].copy()

In [21]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [23]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

In [26]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [27]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [29]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.833333,0.511111,0.586592,0.863636,0.621622,0.823647,0.310345,0.400000,0.494220,...,0.140052,0.486911,0.808824,LAC,0.847222,1.0,2020,2020-03-01,False,0
1,0.0,0.733333,0.088889,0.829609,0.500000,0.108108,0.931864,1.000000,1.000000,0.718208,...,0.031414,0.455497,0.852941,PHI,0.763889,0.0,2020,2020-03-01,True,0
2,0.0,0.400000,0.333333,0.298883,0.227273,0.297297,0.298597,0.413793,0.428571,0.656069,...,0.015707,0.036649,0.264706,CHO,0.138889,1.0,2020,2020-03-01,True,0
3,0.0,0.366667,0.533333,0.164804,0.272727,0.405405,0.304609,0.172414,0.114286,0.855491,...,0.236911,0.104712,0.147059,MIL,0.250000,0.0,2020,2020-03-01,False,1
4,0.0,0.700000,0.488889,0.483240,0.409091,0.540541,0.396794,0.275862,0.371429,0.468208,...,0.086387,0.068063,0.352941,MIN,0.222222,1.0,2020,2020-03-01,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.0,0.766667,0.377778,0.620112,0.818182,0.594595,0.799599,0.310345,0.428571,0.449422,...,0.303665,0.282723,0.779412,POR,0.527778,0.0,2022,2021-10-31,True,2
274,0.0,0.800000,0.000000,1.000000,0.500000,0.135135,0.887776,0.206897,0.285714,0.458092,...,0.123037,1.000000,0.661765,DET,0.222222,0.0,2022,2021-10-31,True,2
275,0.0,0.500000,0.533333,0.282123,0.227273,0.432432,0.234469,0.413793,0.400000,0.710983,...,0.132199,0.649215,0.426471,DAL,0.416667,1.0,2022,2021-10-31,False,2
276,0.0,0.566667,0.400000,0.416201,0.454545,0.459459,0.494990,0.310345,0.400000,0.494220,...,0.041885,0.261780,0.426471,SAC,0.333333,0.0,2022,2021-10-31,True,2


In [30]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [31]:
predictors = list(selected_columns[sfs.get_support()])

In [35]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    seasons = sorted(data["season"].unique())
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        model.fit(train[predictors], train["target"])
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [36]:
predictions = backtest(df, rr, predictors)

In [37]:
predictions

,actual,prediction
92,0,1
93,0,1
94,1,1
95,1,0
96,1,1
...,...,...
273,2,1
274,2,1
275,2,1
276,2,1


In [39]:
from sklearn.metrics import accuracy_score
predictions = predictions[predictions["actual"] !=2]
accuracy_score(predictions["actual"], predictions["prediction"])

0.6025641025641025

In [40]:
df.groupby("home").apply(lambda x: x[x["won"]==1].shape[0] / x.shape[0])

/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/1073487724.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("home").apply(lambda x: x[x["won"]==1].shape[0] / x.shape[0])


home
0.0    0.467626
1.0    0.532374
dtype: float64

In [41]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [43]:
def find_team_averages(team):
    rolling = team.rolling(10).mean()
    return rolling
    
df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

DataError: Cannot aggregate non-numeric type: object

In [44]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,0.0,0.833333,0.511111,0.586592,0.863636,0.621622,0.823647,0.310345,0.400000,0.494220,...,0.095,0.288154,0.140052,0.486911,0.808824,0.847222,1.0,False,PHI,2020
1,0.0,0.733333,0.088889,0.829609,0.500000,0.108108,0.931864,1.000000,1.000000,0.718208,...,0.064,0.199573,0.031414,0.455497,0.852941,0.763889,0.0,True,LAC,2020
2,0.0,0.400000,0.333333,0.298883,0.227273,0.297297,0.298597,0.413793,0.428571,0.656069,...,0.092,0.085379,0.015707,0.036649,0.264706,0.138889,1.0,True,MIL,2020
3,0.0,0.366667,0.533333,0.164804,0.272727,0.405405,0.304609,0.172414,0.114286,0.855491,...,0.089,0.466382,0.236911,0.104712,0.147059,0.250000,0.0,False,CHO,2020
4,0.0,0.700000,0.488889,0.483240,0.409091,0.540541,0.396794,0.275862,0.371429,0.468208,...,0.077,0.288154,0.086387,0.068063,0.352941,0.222222,1.0,True,DAL,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.0,0.766667,0.377778,0.620112,0.818182,0.594595,0.799599,0.310345,0.428571,0.449422,...,0.114,0.230523,0.303665,0.282723,0.779412,0.527778,0.0,True,CHO,2022
274,0.0,0.800000,0.000000,1.000000,0.500000,0.135135,0.887776,0.206897,0.285714,0.458092,...,0.166,0.329776,0.123037,1.000000,0.661765,0.222222,0.0,True,BRK,2022
275,0.0,0.500000,0.533333,0.282123,0.227273,0.432432,0.234469,0.413793,0.400000,0.710983,...,0.029,0.098186,0.132199,0.649215,0.426471,0.416667,1.0,False,SAC,2022
276,0.0,0.566667,0.400000,0.416201,0.454545,0.459459,0.494990,0.310345,0.400000,0.494220,...,0.054,0.199573,0.041885,0.261780,0.426471,0.333333,0.0,True,DAL,2022


In [45]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [46]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10
0,0.0,0.833333,0.511111,0.586592,0.863636,0.621622,0.823647,0.310345,0.400000,0.494220,...,0.095,0.288154,0.140052,0.486911,0.808824,0.847222,1.0,False,PHI,2020
1,0.0,0.733333,0.088889,0.829609,0.500000,0.108108,0.931864,1.000000,1.000000,0.718208,...,0.064,0.199573,0.031414,0.455497,0.852941,0.763889,0.0,True,LAC,2020
2,0.0,0.400000,0.333333,0.298883,0.227273,0.297297,0.298597,0.413793,0.428571,0.656069,...,0.092,0.085379,0.015707,0.036649,0.264706,0.138889,1.0,True,MIL,2020
3,0.0,0.366667,0.533333,0.164804,0.272727,0.405405,0.304609,0.172414,0.114286,0.855491,...,0.089,0.466382,0.236911,0.104712,0.147059,0.250000,0.0,False,CHO,2020
4,0.0,0.700000,0.488889,0.483240,0.409091,0.540541,0.396794,0.275862,0.371429,0.468208,...,0.077,0.288154,0.086387,0.068063,0.352941,0.222222,1.0,True,DAL,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.0,0.766667,0.377778,0.620112,0.818182,0.594595,0.799599,0.310345,0.428571,0.449422,...,0.114,0.230523,0.303665,0.282723,0.779412,0.527778,0.0,True,CHO,2022
274,0.0,0.800000,0.000000,1.000000,0.500000,0.135135,0.887776,0.206897,0.285714,0.458092,...,0.166,0.329776,0.123037,1.000000,0.661765,0.222222,0.0,True,BRK,2022
275,0.0,0.500000,0.533333,0.282123,0.227273,0.432432,0.234469,0.413793,0.400000,0.710983,...,0.029,0.098186,0.132199,0.649215,0.426471,0.416667,1.0,False,SAC,2022
276,0.0,0.566667,0.400000,0.416201,0.454545,0.459459,0.494990,0.310345,0.400000,0.494220,...,0.054,0.199573,0.041885,0.261780,0.426471,0.333333,0.0,True,DAL,2022


In [47]:
df = df.dropna()

In [48]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10
0,0.0,0.833333,0.511111,0.586592,0.863636,0.621622,0.823647,0.310345,0.400000,0.494220,...,0.095,0.288154,0.140052,0.486911,0.808824,0.847222,1.0,False,PHI,2020
1,0.0,0.733333,0.088889,0.829609,0.500000,0.108108,0.931864,1.000000,1.000000,0.718208,...,0.064,0.199573,0.031414,0.455497,0.852941,0.763889,0.0,True,LAC,2020
2,0.0,0.400000,0.333333,0.298883,0.227273,0.297297,0.298597,0.413793,0.428571,0.656069,...,0.092,0.085379,0.015707,0.036649,0.264706,0.138889,1.0,True,MIL,2020
3,0.0,0.366667,0.533333,0.164804,0.272727,0.405405,0.304609,0.172414,0.114286,0.855491,...,0.089,0.466382,0.236911,0.104712,0.147059,0.250000,0.0,False,CHO,2020
4,0.0,0.700000,0.488889,0.483240,0.409091,0.540541,0.396794,0.275862,0.371429,0.468208,...,0.077,0.288154,0.086387,0.068063,0.352941,0.222222,1.0,True,DAL,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.0,0.766667,0.377778,0.620112,0.818182,0.594595,0.799599,0.310345,0.428571,0.449422,...,0.114,0.230523,0.303665,0.282723,0.779412,0.527778,0.0,True,CHO,2022
274,0.0,0.800000,0.000000,1.000000,0.500000,0.135135,0.887776,0.206897,0.285714,0.458092,...,0.166,0.329776,0.123037,1.000000,0.661765,0.222222,0.0,True,BRK,2022
275,0.0,0.500000,0.533333,0.282123,0.227273,0.432432,0.234469,0.413793,0.400000,0.710983,...,0.029,0.098186,0.132199,0.649215,0.426471,0.416667,1.0,False,SAC,2022
276,0.0,0.566667,0.400000,0.416201,0.454545,0.459459,0.494990,0.310345,0.400000,0.494220,...,0.054,0.199573,0.041885,0.261780,0.426471,0.333333,0.0,True,DAL,2022


In [50]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/wz/xjbtmqjs7d1820qtqz_shp7r0000gn/T/ipykernel_13397/1688004556.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(

In [52]:
df = df.copy()

In [54]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], 
                left_on=["team", "date_next"], 
                right_on=["team_opp_next", "date_next"])

In [55]:
full

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
0,0.0,0.833333,0.511111,0.586592,0.863636,0.621622,0.823647,0.310345,0.400000,0.494220,...,0.204188,0.115183,0.500000,0.194444,1.0,True,ATL,2021,PHI,ATL
1,0.0,0.733333,0.088889,0.829609,0.500000,0.108108,0.931864,1.000000,1.000000,0.718208,...,0.086387,0.068063,0.352941,0.222222,1.0,True,DAL,2020,LAC,DAL
2,0.0,0.700000,0.488889,0.483240,0.409091,0.540541,0.396794,0.275862,0.371429,0.468208,...,0.031414,0.455497,0.852941,0.763889,0.0,True,LAC,2020,DAL,LAC
3,0.0,0.200000,0.266667,0.139665,0.454545,0.405405,0.533066,0.379310,0.428571,0.586705,...,0.124346,0.293194,0.470588,0.138889,0.0,True,PHO,2021,LAL,PHO
4,1.0,0.900000,0.711111,0.519553,0.818182,0.648649,0.757515,0.793103,0.742857,0.774566,...,0.430628,0.738220,0.647059,1.000000,1.0,False,POR,2021,DEN,POR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,0.0,0.566667,0.177778,0.572626,0.318182,0.189189,0.513026,0.689655,0.542857,0.942197,...,0.083770,0.146597,0.000000,0.430556,1.0,False,DAL,2022,SAC,DAL
234,0.0,0.000000,0.133333,0.000000,0.272727,0.459459,0.278557,0.586207,0.600000,0.679191,...,0.071990,0.329843,0.470588,0.472222,1.0,True,SAC,2022,DAL,SAC
235,0.0,0.433333,0.377778,0.304469,0.545455,0.621622,0.498998,0.689655,0.685714,0.710983,...,0.064136,0.151832,0.397059,0.319444,0.0,True,BRK,2022,DET,BRK
236,0.0,0.433333,0.377778,0.304469,0.363636,0.513514,0.358717,0.241379,0.285714,0.549133,...,0.096859,0.366492,0.294118,0.444444,1.0,False,UTA,2022,MIL,UTA


In [56]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,PHI,ATL,ATL,PHI,2021-06-06
1,LAC,DAL,DAL,LAC,2021-06-02
2,DAL,LAC,LAC,DAL,2021-06-02
3,LAL,PHO,PHO,LAL,2021-06-03
4,DEN,POR,POR,DEN,2021-06-03
...,...,...,...,...,...
233,SAC,DAL,DAL,SAC,2021-10-31
234,DAL,SAC,SAC,DAL,2021-10-31
235,DET,BRK,BRK,DET,2021-10-31
236,MIL,UTA,UTA,MIL,2021-10-31


In [57]:
removed_columns = list(full.columns[full.dtypes == "object"])

In [58]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [59]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [61]:
predictors = list(selected_columns[sfs.get_support()])

In [64]:
predictions = backtest(full, rr, predictors)

In [65]:
accuracy_score(predictions["actual"], predictions["prediction"])

1.0